In [ ]:
import numpy as np
import cv2,os
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_path = r'C:\Users\91727\Desktop\dataset'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))
print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [ ]:
img_size = 100
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)           
        resized = cv2.resize(gray, (img_size, img_size))
        data.append(resized)
        target.append(label_dict[category])            

In [ ]:
data = np.array(data)/255.0
data = np.reshape(data, (data.shape[0], img_size, img_size,1))
target = np.array(target)
target = np_utils.to_categorical(target)

In [ ]:
train_data,test_data,train_target,test_target = train_test_split(data, target, test_size = 0.1)

In [ ]:
dataGen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
dataGen.fit(train_data)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Conv2D(250, (3,3), input_shape = data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(100, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

In [ ]:
model.compile(Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit_generator(dataGen.flow(train_data, train_target, batch_size = 20), epochs = 20)

In [ ]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
source = cv2.VideoCapture(0)
labels_dict = {0:'NO MASK', 1:'MASK'}
color_dict = {0:(0,0,255), 1:(0,255,0)}

In [ ]:
while(True):

    ret,img = source.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray,1.3,5)  

    for x,y,w,h in faces:
    
        face_img = gray[y:y+w,x:x+w]
        resized = cv2.resize(face_img, (100,100))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, (1,100,100,1))
        result = model.predict(reshaped)

        label = np.argmax(result, axis=1)[0]
        maxprob = np.amax(result)
        if maxprob>0.75:
            cv2.rectangle(img, (x,y), (x+w,y+h), color_dict[label], 2)
            cv2.rectangle(img, (x,y-40), (x+w,y), color_dict[label], -1)
            cv2.putText(img, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(img, str(maxprob*100) + "%", (x,y-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
        
    cv2.imshow('LIVE', img)
    key = cv2.waitKey(1)
    
    if(key == 27):
        break
        
cv2.destroyAllWindows()
source.release()